### Read In Data

In [25]:
import json
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#from transformers import AutoTokenizer #kills my kernal

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jackieglasheen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jackieglasheen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jackieglasheen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

punctuation
stop word
limit to first X words
append tilile
lematize

In [4]:
path = '../data/articles.json'

# Read the JSON file
with open(path, 'r') as file:
    data = json.load(file)



In [5]:
# Print a sample of data
num_rows= 2  
for i in range(num_rows):
    print(data[i])



{'title': '\n\t\tTrump DC hotel falls millions short of projected revenue in first months\t', 'article_text': 'President Trump’s Washington, D.C., hotel made at least $2 million less than projected in its first two months of operation, according to House Democrats citing documents obtained from the General Services Administration (GSA).Democrats on the House Oversight and Transportation committees revealed the information in a letter sent Monday to GSA Administrator Timothy Horne, in which they asked how the agency planned to handle the alleged breach of the lease on the hotel housed in the Old Post Office, just east of the White House on Pennsylvania Avenue.A provision in the lease, signed years before Trump began his presidential campaign, prevents any federal elected official from being included on or benefiting from the lease. The Democrats claim in their letter that Trump’s ascendance to the presidency on Friday is a breach of the lease.The documents Democrats obtained from the GS

In [26]:
#development sample
data_sample = data[0:30]

with open("../data/30_sample_articles.json", "w") as fp:
    json.dump(data_sample , fp) 

In [27]:
for article in data_sample:
    for key in article:
        print(key)
    break

title
article_text
images
uuid
parent


In [28]:
data[300].keys()

dict_keys(['title', 'article_text', 'images', 'uuid', 'parent'])

Looks like we could either grab sourse from image url main or imag url social

In [29]:
for sample in data_sample:
    #print(sample["images"])
    for url_item in sample["images"]:
        print(url_item["image_type"], url_item["url"])
    break



main https://i0.wp.com/thehill.com/wp-content/uploads/sites/2/2017/01/trumphotel618289268.jpg?w=2000&ssl=1
social https://thehill.com/wp-content/uploads/sites/2/2017/01/trumphotel618289268.jpg?w=1280


# Experimenting with preprocessing

In [30]:
# isolate the article text 
#text_only = []
#for article in data_sample:
#    text_only.append(article["article_text"])


In [31]:
json_string = json.dumps(data_sample)
df = pd.read_json(json_string)
df.head()

/var/folders/z3/7313_vxj6t1cpgcl_20j69cm0000gn/T/ipykernel_52964/3507971266.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_string)


,title,article_text,images,uuid,parent
0,\n\t\tTrump DC hotel falls millions short of p...,"President Trump’s Washington, D.C., hotel made...",[{'url': 'https://i0.wp.com/thehill.com/wp-con...,73ff4208-406e-11ee-a96e-33dec8f414a2,73ff4208-406e-11ee-a96e-33dec8f414a2
1,\n\t\tOvernight Tech: Microsoft wants end to f...,MICROSOFT CALLS OUT GOVERNMENT HACKING PRACTIC...,[{'url': 'https://i0.wp.com/thehill.com/wp-con...,77a0c19b-406e-11ee-a96e-33dec8f414a2,77a0c19b-406e-11ee-a96e-33dec8f414a2
2,\n\t\tAnother US motorcycle company considerin...,Minnesota-based Polaris acknowledged on Friday...,[{'url': 'https://thehill.com/wp-content/uploa...,8cfe90c9-406e-11ee-a96e-33dec8f414a2,8cfe90c9-406e-11ee-a96e-33dec8f414a2
3,\n\t\tTrump: ‘I don’t need a summit’ with Mexi...,President Trump said he does not need to hold ...,[{'url': 'https://i0.wp.com/thehill.com/wp-con...,9cdbdae6-406e-11ee-a96e-33dec8f414a2,9cdbdae6-406e-11ee-a96e-33dec8f414a2
4,\n\t\tUS calls emergency meeting of UN nuclear...,The United States called an emergency meeting ...,[{'url': 'https://thehill.com/wp-content/uploa...,a3a5f190-406e-11ee-a96e-33dec8f414a2,a3a5f190-406e-11ee-a96e-33dec8f414a2


# Preprocessing

In [38]:
#concate text and title
df["title_text"] = df["title"] + " " +  df["article_text"]

In [39]:

#lowercase text
df['title_text'] = df['title_text'].str.lower()

# remove some unicode, for tabs and newline and return. 
# are there others?
df['title_text'] = df['title_text'].apply(lambda x: re.sub(r'[\n\t\r]', '', x))

# remove punctuation, replace with space
df['title_text'] = df['title_text'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

In [40]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{"it's", "shouldn't", 'after', 'few', 'having', "she's", 'did', 'just', 'are', 't', 'weren', "you'll", 'under', 'myself', 'but', 'out', "you'd", 'him', 'm', 'haven', 'wouldn', 'with', 'in', 'y', 'other', 'should', 'most', 'from', 'about', 'a', 'very', 'own', 'o', 'hers', 'mustn', 'their', "haven't", 'where', 'himself', 'why', 'd', 'so', 'any', "wouldn't", 'she', 'needn', 'through', 'll', 'while', 'what', 'at', 'no', "doesn't", 'as', 'nor', 'whom', 'couldn', 'is', 'more', 'being', 'into', 'was', 'hadn', 'or', "hasn't", 've', 'ain', 'here', 'not', 'theirs', 'above', 'isn', 'it', 'mightn', 'do', 'aren', "shan't", 'because', "isn't", 'for', 'off', 'then', 'again', "couldn't", 'both', 'has', 'were', 'such', 'me', 'all', 'there', "aren't", 'only', 'on', 'you', 'these', 's', 'yours', 'during', 'against', 'don', "didn't", 'he', "that'll", "hadn't", 'down', 'the', 'if', 'doing', 'how', 'until', 'each', 'doesn', "you're", 'shan', 'too', 'shouldn', 'i', "mightn't", 'her', 'this', 'by', 'them', "m

In [41]:
len(stop_words)

179

In [42]:

def remove_stopwords_and_lemmatize(text):
 
    word_tokens = word_tokenize(text)  
    removed_stopword = [word for word in word_tokens if word not in stop_words]  

    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(token) for token in removed_stopword]
    
    return ' '.join(lemmatized_text[:50])

df['title_text'] = df['title_text'].apply(remove_stopwords_and_lemmatize)


In [43]:
# remove extra spaces
df['title_text_clean'] = df['title_text'].str.replace(r'\s+', ' ', regex=True)

In [44]:
df['title_text_clean'].head()
# US--> U?

0    trump dc hotel fall million short projected re...
1    overnight tech microsoft want end fed secret h...
2    another u motorcycle company considering movin...
3    trump need summit mexico el salvador guatemala...
4    u call emergency meeting un nuclear watchdog g...
Name: title_text_clean, dtype: object

In [51]:
#cleaned sample

#selected_columns = df[['title', 'text', 'title_text_clean']]
df[['title', 'article_text', 'title_text_clean']].to_csv("../data/30_sample_articles_cleaned.csv", index=False)

In [ ]:
# Create Embedded tokens 
# This code needs to be tested

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized_texts = df['title_text_token'].apply(lambda x: tokenizer.tokenize(x))
